In [4]:
from bs4 import BeautifulSoup as bs
import requests
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')

driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)

C:\Users\Chloe\AppData\Local\Temp\ipykernel_24744\3096828039.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)


In [5]:
df = pd.DataFrame({'title': pd.Series(dtype='str'),
                   'release_date': pd.Series(dtype='str'),
                   'rating': pd.Series(dtype='str'),
                   'critic_score': pd.Series(dtype='str'),
                   'audience_score': pd.Series(dtype='str'),
             
})

print(df)

#specifies the website to scrape and use selenium
driver.get("https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc")

WebDriverWait(driver, 1).until(
    lambda s: s.find_element(By.CLASS_NAME,"clamp-list").is_displayed()
)

i = 0
while True:
    try:
        # waits until the head-block class is displayed
        WebDriverWait(driver, 1).until(
            lambda s: s.find_element(By.CLASS_NAME,'clamp-list').is_displayed()
        )
    except TimeoutException:
        break

    url = "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc"
    response = requests.get(url)
    html = response.content
    #creates of beautiful soup object of the current url
    soup = bs(html, "lxml")
    
    time.sleep(1)
    #makes a list of each div with a class of "item-info"
    all_item_info = soup.find_all("td", class_="clamp-summary-wrap")
    #for each item in the list, gather all the relevant data about the books
    for item_info in all_item_info:
        try:
            movie_title = item_info.find("a", class_="title").get("h3").get_text(strip=True)
        except:
            movie_title = pd.NA
        try:
            movie_release = item_info.find("div", class_="clamp-details").find_all("span")[-2].get_text(strip=True)
        except:
            movie_release = pd.NA
        try:
            movie_rating = item_info.find("div", class_="clamp-details").find_all("span")[-1].get_text(strip=True)
        except:
            movie_rating = pd.NA
        try:
            movie_crit_score = item_info.find("div", class_="metascore_anchor").find("div").get_text(strip=True)
        except:
            movie_crit_score = pd.NA
        try:
            movie_user_score = item_info.find("div", class_="browse-score-clamp").find("div", class_="clamp-userscore").find("a").find("div").get_text(strip=True)
        except:
            movie_user_score = pd.NA
            
        df = pd.concat([df,pd.Series({'title': movie_title,
                       'release_date': movie_release,
                       'rating': movie_rating,
                       'critic_score': movie_crit_score,
                       'audience_score': movie_user_score
                      }).to_frame().T], ignore_index=True)
    break
        
df.to_csv('bestsellers.csv')
df  

Empty DataFrame
Columns: [title, release_date, rating, critic_score, audience_score]
Index: []


TooManyRedirects: Exceeded 30 redirects.